In [37]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
import pickle
import tensorflow as tf

In [38]:
### Load the trained model, scaler pickle,onehot
model=load_model('model.h5')
with open('label_encoder_gender.pkl','rb') as fl:
    label_encoder_gender=pickle.load(fl)

with open('open_hot_encoder_geo.pkl','rb') as fl:
    one_hot_encoder_geo=pickle.load(fl)

In [39]:
input_data= {
    'CreditScore':600,
    'Geography':'France',
    'Gender':'Male',
    'Age':40,
    'Tenure':3,
    'Balance':60000,
    'NumOfProducts':2,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':50000
}

In [40]:
geo_encoded=one_hot_encoder_geo.transform([[input_data['Geography']]])
geo_encoded_df=pd.DataFrame(geo_encoded,columns=one_hot_encoder_geo.get_feature_names_out(['Geography']))

/home/workstation-p/anaconda3/envs/conda-env-3-12/lib/python3.12/site-packages/sklearn/utils/validation.py:2691: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [41]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [42]:
input_df=pd.DataFrame([input_data])

In [43]:
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [44]:
input_df['Gender']=label_encoder_gender.transform(input_df['Gender'])

In [45]:
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [46]:
input_df=pd.concat([input_df.drop('Geography',axis=1),geo_encoded_df],axis=1)

In [47]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [48]:
with open("scaler.pkl","rb") as fl:
    scaler=pickle.load(fl)

In [49]:
input_scaled=scaler.transform(input_df)

In [50]:
input_scaled

array([[-0.54007722,  0.91186722,  0.10685843, -0.69164931, -0.26034447,
         0.81311138,  0.64946204,  0.96978946, -0.88017893,  1.00053348,
        -0.57776083, -0.57735027]])

In [52]:
prediction_prob=model.predict(input_scaled)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step


In [55]:
print(prediction_prob[0][0])

0.046234664


In [56]:
if(prediction_prob[0][0]>0.5):
    print('The customer is likely to churn')
else:
    print('The customer is not likely to churn.')

The customer is not likely to churn.
